<a href="https://colab.research.google.com/github/AdryGP/ABP-FindMo/blob/main/FindMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FindMo: Manual de Uso

---



In [28]:
!pip install streamlit

     |████████████████████████████████| 7.4MB 2.6MB/s 
     |████████████████████████████████| 102kB 10.2MB/s 
     |████████████████████████████████| 4.5MB 43.0MB/s 
     |████████████████████████████████| 6.7MB 42.0MB/s 
     |████████████████████████████████| 163kB 37.4MB/s 
     |████████████████████████████████| 102kB 7.2MB/s 
     |████████████████████████████████| 112kB 588kB/s 
     |████████████████████████████████| 71kB 3.7MB/s 
     |████████████████████████████████| 122kB 34.8MB/s 
     |████████████████████████████████| 133kB 40.4MB/s 
     |████████████████████████████████| 71kB 5.0MB/s 
  Created wheel for boto3: filename=boto3-1.16.13-py2.py3-none-any.whl size=128453 sha256=503f062c59e4f98196773f70d5c69c7b34c0291d401d00f5f8bd029467f8f2d5
  Stored in directory: /root/.cache/pip/wheels/bd/ca/cc/d99cff66806b87034af25f8fd8b0adb3e0151b17eea7891143
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=d0ddcb7606a3213b9a4093b0b4b31dceceaa74

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2020-11-07 17:24:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.71.209.190, 54.164.152.149, 35.174.46.144, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.71.209.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  54.1MB/s    in 0.2s    

2020-11-07 17:24:30 (54.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


# Paso 1: Carga de Datasets Originales

Primero, cargamos los datasets originales, almacenados en un drive público:

In [2]:
# !pip install gdown

!mkdir -p original_datasets/videogames 

!mkdir -p original_datasets/movies 

!mkdir -p system_datasets

import gdown as gd

# Videojuegos
gd.download('https://drive.google.com/uc?id=1-KgMce1i1IuEEsk-xUnIRTWq1R8jr8Ey', 'original_datasets/videogames/steam_list.csv', quiet=False) # Títulos de videojuegos (Steam)

gd.download('https://drive.google.com/uc?id=1uYAG398bhK7fylytYixJo8xdHirFd_Xe', 'original_datasets/videogames/steam_descriptions.csv', quiet=False) # Descripciones de videojuegos (Steam)

gd.download('https://drive.google.com/uc?id=1IqKNR6cdWJWIDXZSxg-TnRITpFWv2tgM', 'original_datasets/videogames/steam_reviews.csv', quiet=False) # Comentarios de videojuegos. (Steam)

# Películas
gd.download('https://drive.google.com/uc?id=1U7OJMy5wmseeeRLiPhwnXRsHzB0Az3-I', 'original_datasets/movies/imbd_movies.csv', quiet=False) # Títulos y descripciones de películas (IMDb)

#gd.download('https://drive.google.com/uc?id=151yjtQnjfC1PxUiyw1xVXhkmHeBVwL0U', 'original_datasets/movies/metacritic_movies.csv', quiet=False) # Títulos de películas (Metacritic)

gd.download('https://drive.google.com/uc?id=157m8NSM5D7xE-VZ8iqVzNNvQa6rMPf3d', 'original_datasets/movies/metacritic_reviews.csv', quiet=False) # Comentarios de películas (Metacritic)

Downloading...
From: https://drive.google.com/uc?id=1-KgMce1i1IuEEsk-xUnIRTWq1R8jr8Ey
To: /content/original_datasets/videogames/steam_list.csv
5.82MB [00:00, 154MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uYAG398bhK7fylytYixJo8xdHirFd_Xe
To: /content/original_datasets/videogames/steam_descriptions.csv
94.7MB [00:00, 290MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IqKNR6cdWJWIDXZSxg-TnRITpFWv2tgM
To: /content/original_datasets/videogames/steam_reviews.csv
121MB [00:00, 155MB/s]
Downloading...
From: https://drive.google.com/uc?id=1U7OJMy5wmseeeRLiPhwnXRsHzB0Az3-I
To: /content/original_datasets/movies/imbd_movies.csv
47.9MB [00:00, 242MB/s]
Downloading...
From: https://drive.google.com/uc?id=157m8NSM5D7xE-VZ8iqVzNNvQa6rMPf3d
To: /content/original_datasets/movies/metacritic_reviews.csv
51.7MB [00:00, 247MB/s]


'original_datasets/movies/metacritic_reviews.csv'

# Paso 2: Confección de Datasets Propios seleccionando un subconjuntos de los Datasets Originales

Abrimos los ficheros con Pandas:

In [3]:
import pandas as pd

# Videojuegos
originalSteamList = pd.read_csv('original_datasets/videogames/steam_list.csv')
originalSteamDescriptions = pd.read_csv('original_datasets/videogames/steam_descriptions.csv')
originalSteamReviews = pd.read_csv('original_datasets/videogames/steam_reviews.csv')

# Películas
originalIMBDMoviesWithDescription = pd.read_csv('original_datasets/movies/imbd_movies.csv')
#originalMetacriticMovies = pd.read_csv('original_datasets/movies/metacritic_movies.csv')
originalMetacriticReviews = pd.read_csv('original_datasets/movies/metacritic_reviews.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Reducimos el tamaño del dataset de videojuegos, solo considerando aquellos juegos que han vendido más de 20k unidades

In [6]:
originalSteamList = originalSteamList[originalSteamList['owners'] != '0-20000']

Reducimos el tamaño del dataset de cine, solo considerando películas con más de 70 reseñas por usuarios

In [7]:
originalIMBDMoviesWithDescription = originalIMBDMoviesWithDescription[originalIMBDMoviesWithDescription['reviews_from_users'] > 70]

Ahora, unimos datasets y seleccionamos las columnas necesarias:

In [8]:
# Estas líneas crean una copia del csv con un id para cada elemento con el formato que hemos especificado (<inicialMedio><autoIncremental>):
# Videojuegos
listadoVideojuegos = originalSteamList.copy()
listadoVideojuegos.insert(0, 'id_obra', range(0, len(listadoVideojuegos)))
listadoVideojuegos = listadoVideojuegos.assign(id_obra=lambda x: 'V'+x.id_obra.astype(str))

# Películas
listadoDescripcionCine = originalIMBDMoviesWithDescription.copy()
listadoDescripcionCine.insert(0, 'id_obra', range(0, len(listadoDescripcionCine)))
listadoDescripcionCine = listadoDescripcionCine.assign(id_obra=lambda x: 'C'+x.id_obra.astype(str))


# Creamos el listado que contiene títulos y descripciones (y guardamos el csv):
# Videojuegos (Descripciones):
listadoDescripcionVideojuegos = pd.merge(listadoVideojuegos, originalSteamDescriptions, left_on='appid', right_on='steam_appid')

listadoDescripcionVideojuegos = listadoDescripcionVideojuegos.rename(columns={'name':'nombre', 'short_description':'descripcion'})

listadoDescripcionVideojuegos = listadoDescripcionVideojuegos.assign(medio='V')

# Almacenamos el listado obtenido en un fichero CSV:
listadoDescripcionVideojuegos[['id_obra', 'medio', 'nombre', 'descripcion']].to_csv('system_datasets/obras_descripcion.csv', index=False)

listadoDescripcionVideojuegos[['id_obra', 'medio', 'nombre', 'descripcion']]

,id_obra,medio,nombre,descripcion
0,V0,V,Counter-Strike,Play the world's number 1 online action game. ...
1,V1,V,Team Fortress Classic,One of the most popular online action games of...
2,V2,V,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...
3,V3,V,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...
4,V4,V,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...
...,...,...,...,...
8474,V8474,V,BLOCKADE Classic,BLOCKADE Classic! First Person Shooter in an e...
8475,V8475,V,Arena Stars,"Choose your Hero, build your Team and take on ..."
8476,V8476,V,Burning Daylight,"Waking up naked in a slaughterhouse, you must ..."
8477,V8477,V,KillStreak.tv,KILLSTREAK.tv is a quick fire 3rd-person multi...


In [9]:
# Cine (Descripciones):
listadoDescripcionCine = listadoDescripcionCine.rename(columns={'title':'nombre', 'description':'descripcion'})
listadoDescripcionCine = listadoDescripcionCine.assign(medio='C')

# Almacenamos el listado obtenido en un fichero CSV (se concatena al final del mismo)
listadoDescripcionCine[['id_obra', 'medio', 'nombre', 'descripcion']].dropna().to_csv('system_datasets/obras_descripcion.csv', index=False, mode='a', header=False)
listadoDescripcionCine[['id_obra', 'medio', 'nombre', 'descripcion']].dropna()


,id_obra,medio,nombre,descripcion
47,C0,C,Nascita di una nazione,The Stoneman family finds its friendship with ...
76,C1,C,Intolerance,"The story of a poor young woman, separated by ..."
147,C2,C,Giglio infranto,"A frail waif, abused by her brutal boxer fathe..."
165,C3,C,Il gabinetto del dottor Caligari,"Hypnotist Dr. Caligari uses a somnambulist, Ce..."
190,C4,C,Dr. Jekyll e Mr. Hyde,Dr. Henry Jekyll experiments with scientific m...
...,...,...,...,...
85811,C9130,C,Falling Inn Love - Ristrutturazione con amore,When city girl Gabriela spontaneously enters a...
85817,C9131,C,Paranormal Investigation,When a young man becomes possessed after playi...
85837,C9132,C,VFW,A group of old war veterans put their lives on...
85839,C9133,C,Coffee & Kareem,Twelve-year-old Kareem Manning hires a crimina...


In [10]:
# Creamos y almacenamos el listado que contiene los comentarios asociados a cada obra:
# Videojuegos (Comentarios):
listadoComentariosVideojuegos = pd.merge(listadoDescripcionVideojuegos, originalSteamReviews, left_on='nombre', right_on='title')

listadoComentariosVideojuegos = listadoComentariosVideojuegos.rename(columns={'review':'comentario', 'recommendation':'valoracion'})

videojuegosPositivos = listadoComentariosVideojuegos[listadoComentariosVideojuegos['valoracion'] == 'Recommended'].copy()
videojuegosPositivos['valoracion'] = '+'

videojuegosNegativos = listadoComentariosVideojuegos[listadoComentariosVideojuegos['valoracion'] == 'Not Recommended'].copy()
videojuegosNegativos['valoracion'] = '-'

listadoComentariosVideojuegos = pd.concat([videojuegosPositivos, videojuegosNegativos])

# Almacenamos el listado obtenido en un fichero CSV:
listadoComentariosVideojuegos[['id_obra', 'comentario', 'valoracion']].to_csv('system_datasets/obras_comentarios.csv', index=False)

listadoComentariosVideojuegos[['id_obra', 'comentario', 'valoracion']]

,id_obra,comentario,valoracion
0,V21,I used to play this game a lot back when it ca...,+
1,V21,Product received for free. 'No hope. No cure. ...,+
2,V21,cool,+
3,V21,"Well ya see ""TAAAAAAAAAAAAAANK""",+
4,V21,Nice game it's a pity that Gaben doesn't creat...,+
...,...,...,...
434176,V8255,The amount of control you have with the graphi...,-
434580,V8255,bad remake mutilates the original work,-
434754,V8255,This game does a lot of things right it releas...,-
434799,V8255,This game sucked,-


In [11]:
# Cine (Comentarios):
listadoComentariosCine = pd.merge(listadoDescripcionCine, originalMetacriticReviews, left_on='nombre', right_on='movie_title')

listadoComentariosCine = listadoComentariosCine.rename(columns={'text':'comentario', 'individual_meta_score':'valoracion'})

cinePositivos = listadoComentariosCine[listadoComentariosCine['valoracion'] > 60].copy()
cinePositivos['valoracion'] = '+'

cineNegativos = listadoComentariosCine[listadoComentariosCine['valoracion'] < 40].copy()
cineNegativos['valoracion'] = '-'

cineNeutrales = listadoComentariosCine[(listadoComentariosCine['valoracion'] >= 40) & (listadoComentariosCine['valoracion'] <= 60)].copy()
cineNeutrales['valoracion'] = '~'

listadoComentariosCine = pd.concat([cinePositivos, cineNeutrales, cineNegativos])

# Almacenamos el listado obtenido en un fichero CSV (se concatena al final del mismo):
listadoComentariosCine[['id_obra', 'comentario', 'valoracion']].dropna().to_csv('system_datasets/obras_comentarios.csv', index=False, mode='a', header=False)

listadoComentariosCine[['id_obra', 'comentario', 'valoracion']].dropna()

,id_obra,comentario,valoracion
0,C1,Probably the most influential of all silent fi...,+
1,C1,"Intolerance may not be perfect, but with such ...",+
2,C1,It may seem flawed in a number of ways to some...,+
3,C1,"As a medium for expressing art, moving picture...",+
4,C1,"Intolerance is thrilling and vital, a collisio...",+
...,...,...,...
64089,C9061,This movie is ridiculous.,-
64090,C9061,Unplanned will make you writhe in agony over h...,-
64091,C9061,There’s not a single scene that speaks to char...,-
64103,C9079,"A lifeless, tone-deaf variation on Invasion of...",-


# Paso 3: Subsistema de Recomendación

## Paso 3.1: Preprocesado del Dataset Propio de obras con descripciones

### Paso 3.1.1: **Tokenizado**, eliminación de **Stopwords** y **Stemming**:

In [12]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedText = []

originalData = pd.read_csv('system_datasets/obras_descripcion.csv')

for row in originalData.itertuples():
    
    
    text = word_tokenize(row[4]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalpha()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData = originalData
preprocessedData['processed_text'] = preprocessedText

preprocessedData

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,id_obra,medio,nombre,descripcion,processed_text
0,V0,V,Counter-Strike,Play the world's number 1 online action game. ...,play world number onlin action game engag incr...
1,V1,V,Team Fortress Classic,One of the most popular online action games of...,one popular onlin action game time team fortre...
2,V2,V,Day of Defeat,Enlist in an intense brand of Axis vs. Allied ...,enlist intens brand axi alli teamplay set wwii...
3,V3,V,Deathmatch Classic,Enjoy fast-paced multiplayer gaming with Death...,enjoy multiplay game deathmatch classic dmc va...
4,V4,V,Half-Life: Opposing Force,Return to the Black Mesa Research Facility as ...,return black mesa research facil one militari ...
...,...,...,...,...,...
17592,C9130,C,Falling Inn Love - Ristrutturazione con amore,When city girl Gabriela spontaneously enters a...,when citi girl gabriela spontan enter contest ...
17593,C9131,C,Paranormal Investigation,When a young man becomes possessed after playi...,when young man becom possess play ouija game g...
17594,C9132,C,VFW,A group of old war veterans put their lives on...,A group old war veteran put live line defend y...
17595,C9133,C,Coffee & Kareem,Twelve-year-old Kareem Manning hires a crimina...,kareem man hire crimin scare mom new boyfriend...


### Paso 3.1.2: Creación de la **Bolsa de Palabras**

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
print("Finished")

Finished


In [14]:
textsBoW.shape

(17597, 19569)

## Paso 3.2: Cálculo de la **Matriz de Distancias**:

In [15]:
from sklearn.metrics import pairwise_distances

distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='cosine')

In [16]:
print(distance_matrix.shape)

(17597, 17597)


## Paso 3.3: Ejemplo de uso 

In [17]:
searchTitle = "Portal" # Obra base para las recomendaciones (Otro ejemplo: "The Elder Scrolls V: Skyrim")
indexOfTitle = preprocessedData[preprocessedData['nombre']==searchTitle].index.values[0]
indexOfTitle

17

In [ ]:
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))

In [ ]:
ordered_scores = sorted(distance_scores, key=lambda x: x[1])

In [ ]:
top_scores = ordered_scores[1:11]
#top_scores

In [ ]:
top_indexes = [i[0] for i in top_scores]
#top_indexes

In [ ]:
top = preprocessedData[['id_obra', 'medio', 'nombre', 'descripcion']].iloc[top_indexes]
#top = top[top['medio'] != 'V'] # Recomendaciones que no sean videojuegos 
top.head(10)

,id_obra,medio,nombre,descripcion
7586,V7586,V,Bridge Constructor Portal,Enter the Aperture Science Enrichment Center a...
12531,C4060,C,Versus,There are 666 portals that connect this world ...
6866,V6866,V,Artifact,A collaboration between legendary game designe...
5292,V5292,V,The Lab,"Welcome to The Lab, a compilation of Valve’s r..."
11625,C3153,C,Essere John Malkovich,A puppeteer discovers a portal that leads lite...
3211,V3211,V,Horizon Shift,Horizon Shift is an explosive wave based singl...
8054,V8054,V,Supraland,"Try the demo! A mix between Portal, Zelda and ..."
10753,C2279,C,La storia infinita 2,A young boy with a distant father enters a wor...
11167,C2694,C,Nightmare - Nuovo incubo,A demonic force has chosen Freddy Krueger as i...
3605,V3605,V,Inbetween Land,Open the portal to a strange world and save yo...


# Paso 4: Integración con Streamlit

## Paso 4.1: Instalación de recursos

In [ ]:
!pip install streamlit

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [ ]:
!unzip ngrok-stable-linux-amd64.zip

## Paso 4.2: Creacion del tunerl con ngrok

In [73]:
get_ipython().system_raw('./ngrok http 8501 &')

Es necesario ejecutar todo el notebook antes de acceder al enlace proporcionado por la siguiente linea de codigo:

In [74]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Es importante ejecutar las proximas celdas, una vez ejecutadas, clickar en el siguiente enlace: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Es importante ejecutar las proximas celdas, una vez ejecutadas, clickar en el siguiente enlace: https://da3e31307446.ngrok.io


In [75]:
%%writefile prueba.py
import os
import streamlit as st
import pandas as pd

@st.cache # Ejecutar sólo una vez:
def getPreprocessedData():

  # PREPROCESADO DATASETS:
  from nltk.tokenize import word_tokenize 
  from nltk.corpus import stopwords
  from nltk.stem import PorterStemmer

  import nltk
  nltk.download('punkt')
  nltk.download('stopwords')

  ps = PorterStemmer()

  preprocessedText = []

  originalData = pd.read_csv('system_datasets/obras_descripcion.csv')

  for row in originalData.itertuples():     
    text = word_tokenize(row[4]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalpha()]
    text = " ".join(text)
    
    preprocessedText.append(text)

  preprocessedData = originalData
  preprocessedData['processed_text'] = preprocessedText

  return preprocessedData


@st.cache # Ejecutar sólo una vez:
def getDistanceMatrix(preprocessedData):
  from sklearn.feature_extraction.text import TfidfVectorizer

  bagOfWordsModel = TfidfVectorizer()
  bagOfWordsModel.fit(preprocessedData['processed_text'])
  textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])



  from sklearn.metrics import pairwise_distances

  distance_matrix = pairwise_distances(textsBoW,textsBoW ,metric='cosine')

  return distance_matrix


# Obtener datos preprocesados y matriz de distancias
preprocessedData = getPreprocessedData()
distance_matrix = getDistanceMatrix(preprocessedData)

def getObrasSimilares(tituloObra):
  searchTitle = tituloObra # Obra base para las recomendaciones
  indexOfTitle = preprocessedData[preprocessedData['nombre']==searchTitle].index.values[0]
  distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
  ordered_scores = sorted(distance_scores, key=lambda x: x[1])
  top_scores = ordered_scores[1:11]
  top_indexes = [i[0] for i in top_scores]
  top = preprocessedData[['id_obra', 'medio', 'nombre', 'descripcion']].iloc[top_indexes]

  return top

# PÁGINA STREAMLIT
# Cargar descripciones obras:
obrasConDescripcion = pd.read_csv('system_datasets/obras_descripcion.csv') 

st.header("FindMo: Encuentra más entretenimiento interesante")
st.write("¡Busca una obra que te guste! Encontraremos otras que te puedan interesar")


seleccion = st.selectbox(
    "Seleccione una obra de las siguientes",
    (
        obrasConDescripcion[['nombre']]
    ),
    0    
)

#Aqui se realizaria el computo que ya tenemos realizado y despues se mostrarian las obras

recomendaciones = getObrasSimilares(seleccion)

st.write('Obras similares a', seleccion, ':')

for index, row in recomendaciones.iterrows():
    st.write(row['nombre'])

st.write("Streamlit permite importar imagenes, puede ser interesante utilizar la caratula en esto")


Overwriting prueba.py


In [ ]:
!streamlit run prueba.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.74.2.155:8501

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
tcmalloc: large alloc 2477236224 bytes == 0x3c068000 @  0x7fe3fe6ab001 0x7fe3f966c765 0x7fe3f96d0bb0 0x7fe3f96d2a4f 0x7fe3f9769048 0x50a4a5 0x50cc96 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x588d41 0x59fd0e 0x50d256 0x507be4 0x588d41 0x59fd0e 0x50d256 0x507be4
